In [69]:
import pickle as pk
import dgl as d
import pandas as pd
import numpy as np
from datasets.dataset import FeatureLookup

In [70]:
class a:
    def __init__(self, node2vec_dim, node2vec_length, node2vec_walk, node2vec_epochs, node2vec_batchsize):
        self.node2vec_dim = node2vec_dim
        self.node2vec_length = node2vec_length
        self.node2vec_walk = node2vec_walk
        self.node2vec_epochs = node2vec_epochs
        self.node2vec_batchsize = node2vec_batchsize
        self.node2vec_windows = 3
        self.slices = 10

# node2vec
def eraser(datasets, args):
    label = None

    # try:
    # label = pk.load(open(f'./pretrain/user_based_{args.slices}.pk', 'rb'))
    # except IOError:

    userg = d.graph([])
    user_lookup = FeatureLookup()
    ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_table.csv')
    ufile = pd.DataFrame(ufile)
    ulines = ufile.to_numpy()

    for i in range(339):
        user_lookup.register('User', i)

    for ure in ulines[:, 2]:
        user_lookup.register('URE', ure)

    for uas in ulines[:, 4]:
        user_lookup.register('UAS', uas)

    userg.add_nodes(len(user_lookup))

    for line in ulines:
        uid = line[0]
        ure = user_lookup.query_id(line[2])

        if not userg.has_edges_between(uid, ure):
            userg.add_edges(uid, ure)

        uas = user_lookup.query_id(line[4])
        if not userg.has_edges_between(uid, uas):
            userg.add_edges(uid, uas)

    userg = d.add_self_loop(userg)
    userg = d.to_bidirected(userg)

    from node2vec import Node2Vec
    G = userg.to_networkx()

    # 设置node2vec参数
    node2vec = Node2Vec(
        G,
        dimensions=args.node2vec_dim,  # 嵌入维度
        p=1,  # 回家参数
        q=0.5,  # 外出参数
        walk_length=args.node2vec_length,  # 随机游走最大长度
        num_walks=args.node2vec_walk,  # 每个节点作为起始节点生成的随机游走个数
        workers=4  # 并行线程数
    )

    # p=1, q=0.5, n_clusters=6。DFS深度优先搜索，挖掘同质社群
    # p=1, q=2, n_clusters=3。BFS宽度优先搜索，挖掘节点的结构功能。

    # 训练Node2Vec，参数文档见 gensim.models.Word2Vec
    model = node2vec.fit(
        window=args.node2vec_windows,  # Skip-Gram窗口大小
        epochs=args.node2vec_epochs,
        min_count=1,  # 忽略出现次数低于此阈值的节点（词）
        batch_words=args.node2vec_batchsize  # 每个线程处理的数据量
    )
    ans = model.wv.vectors[:339]

    df = np.array(ans)

    split_train_Tensor = []
    # 整理特征
    pk.dump(df, open(f'./pretrain/user_embeds.pk', 'wb'))
    # return df

    def k_mean(inputs, k):
        from sklearn.cluster import KMeans
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(inputs)
        return kmeans.labels_

    label = k_mean(df, args.slices)
    pk.dump(label, open(f'./pretrain/user_based_{args.slices}.pk', 'wb'))

    # 检查每个类的个数
    dic = {}
    for i in label:
        if i not in dic:
            dic[i] = 1
        else:
            dic[i] = dic[i] + 1
    print(dic)

    # # 获得切分矩阵
    # for i in range(args.slices):
    #     split_tensor = np.zeros_like(datasets)
    #     Idx = (label == i).nonzero()[0]
    #     for idx in Idx:
    #         split_tensor[idx] = datasets[idx]
    #     split_train_Tensor.append(split_tensor)

    return label

In [71]:
args = a(128, 20, 50, 20, 32)
label = eraser(None, args)

Computing transition probabilities:   0%|          | 0/507 [00:00<?, ?it/s]

{1: 164, 0: 70, 4: 15, 5: 17, 3: 15, 7: 10, 2: 20, 8: 18, 6: 6, 9: 4}


In [72]:
C = [[] for _ in range(args.slices)]
for i in range(args.slices):
    for id, j in enumerate(label):
        if j == i:
            C[i].append(id)
C = np.array(C, dtype = 'object')
C

array([list([1, 2, 4, 5, 6, 8, 9, 10, 11, 17, 18, 32, 33, 35, 39, 41, 43, 44, 46, 50, 64, 67, 72, 73, 74, 75, 76, 78, 79, 87, 92, 96, 97, 99, 101, 105, 109, 111, 117, 119, 121, 122, 131, 138, 140, 147, 154, 156, 157, 158, 166, 170, 184, 186, 212, 228, 237, 240, 246, 250, 253, 263, 264, 266, 277, 282, 297, 310, 326, 332]),
       list([0, 16, 23, 27, 28, 29, 30, 31, 48, 55, 56, 57, 58, 59, 60, 62, 70, 85, 106, 107, 108, 110, 114, 115, 130, 132, 134, 137, 141, 142, 149, 150, 152, 155, 160, 161, 167, 168, 169, 171, 172, 173, 175, 176, 177, 178, 179, 181, 182, 183, 188, 189, 190, 191, 192, 193, 194, 195, 199, 200, 201, 202, 205, 206, 207, 208, 209, 211, 213, 214, 215, 216, 217, 218, 220, 221, 222, 224, 225, 227, 229, 230, 231, 232, 233, 234, 235, 236, 238, 239, 241, 243, 244, 245, 247, 248, 249, 251, 252, 254, 256, 257, 258, 259, 261, 262, 265, 267, 268, 269, 270, 271, 272, 273, 275, 276, 278, 279, 280, 283, 284, 285, 286, 288, 289, 290, 291, 292, 293, 294, 296, 298, 299, 300, 301, 302, 30

In [73]:
def E_score2(value1, value2):
    return np.sum(np.power(value1 - value2, 2))

pretrain = pk.load(open(f'./pretrain/user_embeds.pk', 'rb'))

center_mean_value = []
center_ = []
for i in range(args.slices):
    temp = []
    for j in C[i]:
        temp.append(pretrain[j])
    center_mean_value.append(np.mean(temp, axis = 0).tolist())
    # print(center_mean_value[i])
    center_.append([i, len(C[i]), center_mean_value[i], C[i]])

center_ = np.array(sorted(center_, key = (lambda x : x[1])), dtype = 'object')


vis = [False for _ in range(args.slices)]

set_changed = True

cnt = 0
while set_changed:
    set_changed = False

    for i in range(len(center_)):
        if center_[i][1] >= 40 or vis[i]:
            continue
        min_dis = 1e9
        goal = -1
        for j in range(i + 1, len(center_)):
            if vis[j]:
                continue
            if center_[i][1] + center_[j][1] <= 40:
                now_dis_between_i_and_j = E_score2(np.array(center_[i][2]), np.array(center_[j][2]))
                if now_dis_between_i_and_j < min_dis:
                    min_dis = now_dis_between_i_and_j
                    goal = j
        break

    a = i
    b = goal

    # 定位这两个集合
    a_ = center_[a]
    b_ = center_[b]



    if goal != -1:
        print(f'{a_[0]} 有 {a_[1]} 个')
        print(f'{b_[0]} 有 {b_[1]} 个')
        print(a_[0], b_[0], '合并后 : ', a_[1] + b_[1], '个')
        vis[a_[0]] = True
        vis[b_[0]] = True
        set_changed = True
        cnt += 1

        print('-' * 10)
        for i in center_:
            print(i[0], i[1])

    # 合并这两个集合
    merge_set = a_[3] + b_[3]
    center_mean_value = []
    for i in range(len(merge_set)):
        center_mean_value.append(pretrain[i])
    mean_value_a_and_b = np.mean(center_mean_value, axis = 0).tolist()
    c_ = [min(a_[0], b_[0]), a_[1] + b_[1], mean_value_a_and_b, merge_set]

    # 将合并的集合与剩下的集合整理到一起
    center_next = []
    center_mean_value = []
    for i in range(args.slices):
        temp = []
        # 如果已经合并了，代表遍历过了
        if vis[i]:
            center_mean_value.append(0)
            continue
        for j in C[i]:
            temp.append(pretrain[j])
        center_mean_value.append(np.mean(temp, axis = 0).tolist())
        center_next.append([i, len(C[i]), center_mean_value[i], C[i]])

    vis[min(a_[0], b_[0])] = False
    C[min(a_[0], b_[0])] = merge_set
    # print(c_[0], c_[1], c_[3])
    if goal != -1:
        center_next.append(c_)
        print('-' * 10)

    if goal != -1:
        for i in center_next:
             print(i[0], i[1])

    center_next = np.array(sorted(center_next, key = (lambda x : x[1])), dtype = 'object')
    center_ = center_next
    print('-' * 80)

    # break
print(f'合并了{cnt}次')
center_

9 有 4 个
2 有 20 个
9 2 合并后 :  24 个
----------
9 4
6 6
7 10
3 15
4 15
5 17
8 18
2 20
0 70
1 164
----------
0 70
1 164
3 15
4 15
5 17
6 6
7 10
8 18
2 24
--------------------------------------------------------------------------------
6 有 6 个
2 有 24 个
6 2 合并后 :  30 个
----------
6 6
7 10
3 15
4 15
5 17
8 18
2 24
0 70
1 164
----------
0 70
1 164
3 15
4 15
5 17
7 10
8 18
2 30
--------------------------------------------------------------------------------
7 有 10 个
2 有 30 个
7 2 合并后 :  40 个
----------
7 10
3 15
4 15
5 17
8 18
2 30
0 70
1 164
----------
0 70
1 164
3 15
4 15
5 17
8 18
2 40
--------------------------------------------------------------------------------
3 有 15 个
8 有 18 个
3 8 合并后 :  33 个
----------
3 15
4 15
5 17
8 18
2 40
0 70
1 164
----------
0 70
1 164
2 40
4 15
5 17
3 33
--------------------------------------------------------------------------------
4 有 15 个
5 有 17 个
4 5 合并后 :  32 个
----------
4 15
5 17
3 33
2 40
0 70
1 164
----------
0 70
1 164
2 40
3 33
4 32
-----------------

array([[4, 32,
        list([0.14708498120307922, 0.19849225878715515, 0.13336312770843506, 0.5152713060379028, -0.00397908641025424, -0.28479263186454773, -0.12925691902637482, -0.2779465317726135, -0.110054150223732, 0.39276111125946045, 0.21570754051208496, -0.1577427089214325, 0.27683281898498535, -0.13556186854839325, -0.04102759435772896, 0.7932271957397461, -0.7620140910148621, -0.12107566744089127, -0.7097743153572083, 0.4269057810306549, 0.07651446759700775, 0.215126171708107, -0.28214889764785767, -0.34293973445892334, -0.24253803491592407, -0.04408244788646698, -0.677608072757721, 0.11657074838876724, -0.05550448223948479, -0.32173267006874084, -0.005128085147589445, -0.12631791830062866, -0.323870986700058, -0.17636992037296295, -0.18829025328159332, 0.011994623579084873, 0.32916465401649475, 0.015538807027041912, 0.4632500410079956, 0.36265578866004944, -0.35281404852867126, 0.38289785385131836, 0.14149093627929688, 0.09001568704843521, -0.0885792076587677, 0.2345392554998

In [74]:
len(center_)

5

In [75]:
def k_mean(inputs, k):
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(inputs)
    return kmeans.labels_

In [76]:
extra_set = []
passed = 0
for item in center_:
    passed += 1
    if item[1] >= 40:
        extra_set += item[3]
# extra_set

In [77]:
now_set = []
for i in extra_set:
    now_set.append([i, pretrain[i].tolist()])
now_set = np.array(now_set, dtype = 'object')
inputs = np.array(now_set[:,1].tolist())
now_set

array([[14,
        list([0.290859580039978, 0.12032698094844818, 0.632904589176178, 0.5617689490318298, -0.03511224314570427, -0.5021469593048096, -0.2421434372663498, -0.4344659447669983, 0.4590607285499573, 0.22959288954734802, 0.09707868099212646, 0.14176154136657715, 0.30355510115623474, -0.23205631971359253, 0.11406609416007996, 0.8706274032592773, -0.6479563117027283, -0.635216236114502, -0.5030016899108887, 0.43838760256767273, -0.08899864554405212, 0.694219172000885, -0.17615824937820435, -0.06724908202886581, -0.19237692654132843, 0.4646027684211731, -0.13566751778125763, 0.24780531227588654, 0.2764669358730316, -0.6704236268997192, -0.45564332604408264, -0.5242524743080139, 0.012665756046772003, -0.08847412467002869, -0.14888139069080353, 0.16703437268733978, 0.1955917775630951, 0.09453170746564865, 0.6796432733535767, 0.012598124332726002, -0.3964149057865143, 0.5619091987609863, 0.2995522618293762, -0.07503810524940491, 0.5587028861045837, 0.5866302251815796, 0.02293554879

In [78]:
label = k_mean(inputs, args.slices - passed)

In [79]:
dic = {}
for i in range(len(label)):
    if label[i] not in dic:
        dic[label[i]] = 1
    else:
        dic[label[i]] += 1
dic

{3: 51, 2: 29, 1: 8, 4: 21, 0: 165}

In [80]:
extra_set = []
passed = 0
for item in center_:
    if item[1] >= 40:
        extra_set += item[3]
    else:
        passed += 1

extra_set

[14,
 82,
 94,
 120,
 126,
 129,
 145,
 164,
 180,
 196,
 20,
 37,
 91,
 100,
 128,
 136,
 61,
 69,
 81,
 84,
 15,
 22,
 25,
 26,
 47,
 66,
 68,
 71,
 80,
 88,
 135,
 139,
 165,
 185,
 197,
 203,
 204,
 255,
 260,
 316,
 1,
 2,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 17,
 18,
 32,
 33,
 35,
 39,
 41,
 43,
 44,
 46,
 50,
 64,
 67,
 72,
 73,
 74,
 75,
 76,
 78,
 79,
 87,
 92,
 96,
 97,
 99,
 101,
 105,
 109,
 111,
 117,
 119,
 121,
 122,
 131,
 138,
 140,
 147,
 154,
 156,
 157,
 158,
 166,
 170,
 184,
 186,
 212,
 228,
 237,
 240,
 246,
 250,
 253,
 263,
 264,
 266,
 277,
 282,
 297,
 310,
 326,
 332,
 0,
 16,
 23,
 27,
 28,
 29,
 30,
 31,
 48,
 55,
 56,
 57,
 58,
 59,
 60,
 62,
 70,
 85,
 106,
 107,
 108,
 110,
 114,
 115,
 130,
 132,
 134,
 137,
 141,
 142,
 149,
 150,
 152,
 155,
 160,
 161,
 167,
 168,
 169,
 171,
 172,
 173,
 175,
 176,
 177,
 178,
 179,
 181,
 182,
 183,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 199,
 200,
 201,
 202,
 205,
 206,
 207,
 208,
 209,
 211,
 213,
 214,


In [81]:
ufile = pd.read_csv('./datasets/data/WSDREAM/原始数据/userlist_idx.csv').to_numpy()
inputs = []
for userid in extra_set:
    inputs.append([ufile[userid][2], ufile[userid][4]])
inputs = np.array(inputs)
label = k_mean(inputs, args.slices - passed)

dic = {}
for i in range(len(label)):
    if label[i] not in dic:
        dic[label[i]] = 1
    else:
        dic[label[i]] += 1
dic

{1: 40, 0: 33, 7: 28, 4: 37, 3: 35, 5: 36, 2: 30, 6: 35}

In [82]:
extra_set = np.array(extra_set)
print(extra_set.shape, label.shape)
extra_set = np.concatenate([extra_set.reshape(-1, 1), label.reshape(-1, 1)], axis = 1)
extra_set

(274,) (274,)


array([[ 14,   1],
       [ 82,   0],
       [ 94,   7],
       [120,   0],
       [126,   1],
       [129,   4],
       [145,   3],
       [164,   0],
       [180,   7],
       [196,   5],
       [ 20,   2],
       [ 37,   1],
       [ 91,   4],
       [100,   7],
       [128,   7],
       [136,   5],
       [ 61,   2],
       [ 69,   2],
       [ 81,   5],
       [ 84,   6],
       [ 15,   6],
       [ 22,   5],
       [ 25,   7],
       [ 26,   4],
       [ 47,   0],
       [ 66,   4],
       [ 68,   1],
       [ 71,   3],
       [ 80,   0],
       [ 88,   0],
       [135,   2],
       [139,   6],
       [165,   2],
       [185,   6],
       [197,   6],
       [203,   7],
       [204,   0],
       [255,   5],
       [260,   1],
       [316,   1],
       [  1,   3],
       [  2,   2],
       [  4,   0],
       [  5,   0],
       [  6,   1],
       [  8,   5],
       [  9,   4],
       [ 10,   2],
       [ 11,   3],
       [ 17,   6],
       [ 18,   0],
       [ 32,   2],
       [ 33,

In [83]:
all_set = {}
for i in range(passed):
    all_set[i] = center_[i][3]

for i in all_set:
    print(i, len(all_set[i]))

0 32
1 33


In [84]:
extra_set

array([[ 14,   1],
       [ 82,   0],
       [ 94,   7],
       [120,   0],
       [126,   1],
       [129,   4],
       [145,   3],
       [164,   0],
       [180,   7],
       [196,   5],
       [ 20,   2],
       [ 37,   1],
       [ 91,   4],
       [100,   7],
       [128,   7],
       [136,   5],
       [ 61,   2],
       [ 69,   2],
       [ 81,   5],
       [ 84,   6],
       [ 15,   6],
       [ 22,   5],
       [ 25,   7],
       [ 26,   4],
       [ 47,   0],
       [ 66,   4],
       [ 68,   1],
       [ 71,   3],
       [ 80,   0],
       [ 88,   0],
       [135,   2],
       [139,   6],
       [165,   2],
       [185,   6],
       [197,   6],
       [203,   7],
       [204,   0],
       [255,   5],
       [260,   1],
       [316,   1],
       [  1,   3],
       [  2,   2],
       [  4,   0],
       [  5,   0],
       [  6,   1],
       [  8,   5],
       [  9,   4],
       [ 10,   2],
       [ 11,   3],
       [ 17,   6],
       [ 18,   0],
       [ 32,   2],
       [ 33,

In [85]:
for i in range(passed, passed + len(dic)):
    temp = []
    for j in range(len(extra_set)):
        if extra_set[j][1] == (i - passed):
            temp.append(extra_set[j][0])
    all_set[i] = temp

for i in all_set:
    print(i, len(all_set[i]))

0 32
1 33
2 33
3 40
4 30
5 35
6 37
7 36
8 35
9 28


In [86]:
all_set

{0: [3,
  7,
  42,
  49,
  54,
  125,
  210,
  226,
  242,
  274,
  287,
  295,
  307,
  335,
  338,
  12,
  40,
  45,
  51,
  53,
  89,
  90,
  113,
  116,
  118,
  123,
  143,
  146,
  151,
  153,
  163,
  187],
 1: [13,
  38,
  52,
  83,
  93,
  95,
  98,
  124,
  127,
  133,
  148,
  174,
  198,
  223,
  281,
  19,
  21,
  24,
  34,
  36,
  63,
  65,
  77,
  86,
  102,
  103,
  104,
  112,
  144,
  159,
  162,
  219,
  314],
 2: [82,
  120,
  164,
  47,
  80,
  88,
  204,
  4,
  5,
  18,
  35,
  78,
  237,
  240,
  62,
  85,
  161,
  169,
  172,
  188,
  211,
  213,
  220,
  221,
  225,
  262,
  270,
  283,
  286,
  302,
  303,
  315,
  330],
 3: [14,
  126,
  37,
  68,
  260,
  316,
  6,
  44,
  97,
  117,
  156,
  263,
  266,
  297,
  28,
  56,
  58,
  115,
  178,
  182,
  191,
  209,
  222,
  231,
  232,
  238,
  241,
  247,
  249,
  257,
  265,
  272,
  273,
  275,
  284,
  285,
  292,
  324,
  333,
  334],
 4: [20,
  61,
  69,
  135,
  165,
  2,
  10,
  32,
  33,
  39,
  50,
 

In [87]:
pk.dump(all_set, open('./pretrain/user_based_10.pk', 'wb'))

In [90]:
for i in range(len(all_set)):
    print(i, len(all_set[i]))

0 32
1 33
2 33
3 40
4 30
5 35
6 37
7 36
8 35
9 28


In [89]:
tensor = np.array(pk.load(open('datasets/data/WSDREAM/rt.pk', 'rb')))
split_tensor = []
for i in range(args.slices):
    # idx =
    idx = np.array(C[i], dtype = 'int32')
    temp = np.zeros_like(tensor)
    temp[idx] = tensor[idx]
    split_tensor.append(temp)
np.array(split_tensor).shape

(10, 339, 5825)